# Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the <a href='https://www.cia.gov/the-world-factbook/'>CIA World Factbook</a>, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

* `population` — the global population.
* `population_growth` — the annual population growth rate, as a percentage.
* `area` — the total land and water area.

First, we connect our Jupyter Notebook to the database file `factbook.db`:

In [3]:
%%capture
%load_ext sql
%sql sqlite:///additional_files/factbook.db


***
### Overview of the Data

Now to start, we want to see information on the tables in the database:

In [5]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table'

 * sqlite:///additional_files/factbook.db
   sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


This information tells us that the database contains a table named `facts`.<br>
Next, we want an glips at the first five rows of the `facts` table:

In [6]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///additional_files/factbook.db
   sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

* `name` — the name of the country.
* `area`— the country's total area (both land and water).
* `area_land` — the country's land area in square kilometers.
* `area_water` — the country's waterarea in square kilometers.
* `population` — the country's population.
* `population_growth` — the country's population growth as a percentage.
* `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
* `death_rate` — the country's death rate, or the number of death per year per 1,000 people.



***
### Summary Statistics

Let's start by calculating some summary statistics and look for any outlier countries.

In [8]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
  FROM facts;

 * sqlite:///additional_files/factbook.db
   sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02




A few things stick out from the summary statistics in the last screen:

* There's a country with a population of 0
* There's a country with a population of 7256490011 (or more than 7.2 billion people)

***
### Exploring Outliers

Let's use subqueries to zoom in on just these countries without using the specific values.

In [11]:
%%sql
SELECT name, population
  FROM facts
 WHERE population = (SELECT MIN(population)
                       FROM facts);

 * sqlite:///additional_files/factbook.db
   sqlite:///factbook.db
Done.


name,population
Antarctica,0


It seems like the table contains a row for Antarctica, which explains the population of 0.

In [12]:
%%sql
SELECT name, population
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts);

 * sqlite:///additional_files/factbook.db
   sqlite:///factbook.db
Done.


name,population
World,7256490011



We also see that the table contains a row for the whole world, which explains the maximum population of over 7.2 billion we found earlier.<br>
<br>
Now that we know this, we should recalculate the summary statistics we calculated earlier, while excluding the row for the whole world.


***
### Summary Statistics Revisited

In [17]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
  FROM facts
 WHERE (population != 0) AND (population != (SELECT MAX(population)
                                             FROM facts));

 * sqlite:///additional_files/factbook.db
   sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
48,1367485388,0.0,4.02


There's a country whose population closes in on 1.4 billion!



***
### Exploring Average Population and Area


Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these two columns.

We should take care of discarding the row for the whole planet.


In [19]:
%%sql
SELECT AVG(population) AS avg_pop,
       AVG(area) AS avg_area
  FROM facts
 WHERE name != 'World';

 * sqlite:///additional_files/factbook.db
   sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


We see that the average population is around 32 million and the average area is 555 thousand square kilometers.



***
### Finding Densely Populated Countries

To finish, we'll build on the query we wrote for the previous section to find countries that are densely populated. We'll identify countries that have the following:

* Above-average values for population.
* Below-average values for area.


In [24]:
%%sql
SELECT *
  FROM facts
 WHERE (population > (SELECT AVG(population)
                        FROM facts
                       WHERE name != 'World')) 
   AND (area < (SELECT AVG(area)
                  FROM facts
                 WHERE name != 'World'));

 * sqlite:///additional_files/factbook.db
   sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Some of these countries are generally known to be densely populated, so we have confidence in our results!